# 📘 Week1: GPT-5 챗봇 실습 (Solution)



## 1) 환경 준비 및 단순 출력



In [ ]:
pip install openai ipywidgets

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")

## OpenAI API Quick Start 정답



In [ ]:
def create_greeting(client, model, instructions, user_prompt):
    """Responses API 한 턴 호출 솔루션 구현."""
    response = client.responses.create(
        model=model,
        instructions=instructions,
        input=user_prompt,
    )
    return response.output_text

In [ ]:
greeting = create_greeting(
    client=client,
    model="gpt-5-nano",
    instructions="You are a helpful assistant. Reply in Korean.",
    user_prompt="오늘 하루를 기분 좋게 시작할 수 있는 인사말을 한 문장으로 만들어 주세요.",
)
print(greeting)

## 2) 멀티턴 대화 구성 정답

In [ ]:
def request_chat_completion(
    client,
    model,
    history,
    *,
    reasoning_effort="medium",
    verbosity="medium"
):
    """Responses API 멀티턴 대화 솔루션 구현."""
    history_payload = [msg.copy() for msg in history]

    kwargs = {}
    if reasoning_effort:
        kwargs["reasoning"] = {"effort": reasoning_effort}
    if verbosity:
        kwargs["text"] = {"verbosity": verbosity}

    # API 호출
    response = client.responses.create(
        model=model,
        input=history_payload,
        **kwargs,
    )
    return response.output_text, response

In [ ]:
conversation = [
    {
        "role": "developer",
        "content": "You are a helpful assistant. Reply in Korean and keep answers concise.",
    },
    {
        "role": "user",
        "content": "방금 읽은 책을 한 줄로 요약해 달라고 부탁하는 방법을 알려 줘.",
    },
]

first_text, first_response = request_chat_completion(
    client=client,
    model="gpt-5-mini",
    history=conversation,
    reasoning_effort="medium",
)
print("첫 번째 응답:", first_text)

conversation.append({"role": "assistant", "content": first_text})
conversation.append({"role": "user", "content": "이 내용을 더 친근하게 바꿔 줄래?"})

second_text, second_response = request_chat_completion(
    client=client,
    model="gpt-5-mini",
    history=conversation,
)
print("\n두 번째 응답:", second_text)

## 3) 챗봇 만들기 실습 정답

In [ ]:
from dataclasses import dataclass, field


@dataclass
class ChatSession:
    """Responses API 기반 멀티턴 챗봇 세션 관리 유틸리티 (정답)."""

    client: OpenAI
    model: str = "gpt-5-mini"
    developer_message: str = (
        "You are a helpful assistant. You must answer in Korean and keep answers clear."
    )
    reasoning_effort: str = "medium"
    verbosity: str = "medium"
    history: list = field(default_factory=list)
    last_response_id: object = field(default=None, init=False)

    def __post_init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        """대화 히스토리를 초기 developer 메시지로 리셋합니다."""
        self.history = [
            {
                "role": "developer",
                "content": self.developer_message,
            }
        ]
        self.last_response_id = None

    def send(self, user_message: str):
        """사용자 메시지를 추가하고 모델 응답 텍스트와 response 객체를 반환합니다."""
        cleaned = user_message.strip()
        if not cleaned:
            raise ValueError("user_message는 비어 있을 수 없습니다.")

        self.history.append({"role": "user", "content": cleaned})

        message_text, response = request_chat_completion(
            client=self.client,
            model=self.model,
            history=self.history,
            reasoning_effort=self.reasoning_effort,
        )

        self.history.append({"role": "assistant", "content": message_text})
        self.last_response_id = response.id
        return message_text, response

    def pretty_history(self):
        """현재 히스토리를 읽기 쉬운 문자열로 변환합니다."""
        lines = []
        for msg in self.history:
            prefix = "사용자" if msg["role"] == "user" else "assistant" if msg["role"] == "assistant" else msg["role"]
            lines.append(f"[{prefix}] {msg['content']}")
        return "\n".join(lines)



In [ ]:
chatbot = ChatSession(
    client,
    model="gpt-5-mini",
    developer_message="You are a helpful teaching assistant. Reply in Korean with actionable suggestions.",
    reasoning_effort="low",
)

In [ ]:

reply, raw_response = chatbot.send("Responses API 실습을 복습하는 가장 좋은 방법을 알려 줘.")
print("모델 응답:\n", reply)

print("\n현재 히스토리:\n", chatbot.pretty_history())

In [ ]:
raw_response.usage

## 4) 위젯 기반 챗봇 UI (정답)

In [ ]:
import ipywidgets as widgets
from IPython.display import Markdown, display

chat_ui_session = ChatSession(
    client,
    model="gpt-5-mini",
    developer_message="You are a supportive study buddy. Reply in Korean with friendly tips.",
    reasoning_effort="medium",
)

log_output = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px"))
input_box = widgets.Text(
    placeholder="질문을 입력하고 Enter 또는 '전송'을 클릭하세요.",
    description="질문",
    layout=widgets.Layout(width="100%"),
)
send_button = widgets.Button(description="전송", button_style="primary")
reset_button = widgets.Button(description="초기화")
status_label = widgets.HTML(value="")


def append_message(role: str, text: str) -> None:
    with log_output:
        display(Markdown(f"**{role}**: {text}"))


def handle_send(_=None) -> None:
    text = input_box.value.strip()
    if not text:
        status_label.value = "<span style='color:#d9534f;'>메시지를 입력하세요.</span>"
        return

    status_label.value = ""
    append_message("사용자", text)
    input_box.value = ""
    send_button.disabled = True
    try:
        answer, _ = chat_ui_session.send(text)
        append_message("어시스턴트", answer)
    except Exception as exc:  # pragma: no cover
        append_message("오류", f"{type(exc).__name__}: {exc}")
    finally:
        send_button.disabled = False


def handle_reset(_=None) -> None:
    chat_ui_session.reset()
    log_output.clear_output()
    status_label.value = "<span style='color:#5cb85c;'>세션이 초기화되었습니다.</span>"


send_button.on_click(handle_send)
reset_button.on_click(handle_reset)
input_box.on_submit(handle_send)

controls = widgets.HBox([send_button, reset_button, status_label])
ui = widgets.VBox([input_box, controls, log_output])

display(ui)

